# Initial EDA on Raw UNGA dataset

Data set is taken from here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379

In [510]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from sklearn.preprocessing import scale
%matplotlib inline
import hdbscan
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

In [511]:
df_votes = pd.read_csv('~/Desktop/dataverse_files/UNVotes1-72.tab',sep='\t')

In [512]:
df_votes_year = df_votes.drop(['Country', 'Countryname'], axis=1) 

In [513]:
# Set a DF to only yes and no votes (0 nd 3)
df_votes_year_binary = df_votes_year[(df_votes_year.vote <= 1) | (df_votes_year.vote == 3)]
df_votes_year_binary.head(2)

,rcid,ccode,session,member,vote,year
0,3.0,2,1.0,1.0,1,1946
1,3.0,20,1.0,1.0,3,1946


In [514]:
# I want to turn my 1 = yes and 3 = no into 0 = no and 1 = yes
df_votes_year_binary.vote = df_votes_year_binary.vote.replace(to_replace=3, value=0, inplace=False, limit=None, regex=False, method='pad', axis=None)
df_votes_year_binary.head(2)


/Users/jakerattner/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,rcid,ccode,session,member,vote,year
0,3.0,2,1.0,1.0,1,1946
1,3.0,20,1.0,1.0,0,1946


In [515]:
df_year_features = df_votes_year_binary.drop(['session'], axis=1)

In [516]:
df_year_features.head(2)

,rcid,ccode,member,vote,year
0,3.0,2,1.0,1,1946
1,3.0,20,1.0,0,1946


In [517]:
df_year_features = df_year_features[df_year_features.member == 1]

In [518]:
# NOT SURE THIS CODE WILL BE NEEDED
# df_year_features['ccode'] = df_year_features['ccode'].astype(str)
# df_year_features['year'] = df_year_features['year'].astype(str)
# df_year_features['v_res'] = df_year_features['year'] + "-" + df_year_features['ccode']

In [519]:
df_year_features['v_res'] = df_year_features['rcid']*df_year_features['vote']

In [520]:
df_votes[['ccode','Countryname']].head(2)

,ccode,Countryname
0,2,United States of America
1,20,Canada


In [521]:
df_name_index = pd.DataFrame(df_votes.groupby(['ccode', 'Countryname'])['Countryname'].count())
df_name_index['count'] = df_name_index.Countryname
df_name_index = df_name_index.drop([ 'Countryname','count'], axis=1)
df_name_index.reset_index(inplace=True)
df_name_index.head(2)


,ccode,Countryname
0,2,United States of America
1,20,Canada


# Create function to create DFs for multiple years (1960, 1970, 1980)

In [522]:
df_year_features.head(3)

,rcid,ccode,member,vote,year,v_res
0,3.0,2,1.0,1,1946,3.0
1,3.0,20,1.0,0,1946,0.0
3,3.0,40,1.0,1,1946,3.0


In [523]:
def create_year_df(year):
    df_output_year = df_year_features[df_year_features['year'] == year]
    df_output_year = pd.concat([df_output_year,pd.get_dummies(df_output_year.v_res, prefix='res_')], axis = 1)
    try:
        df_output_year = df_output_year.drop(['year','rcid','member','v_res','vote', 'res__0.0'], axis=1)
    except:
        pass 
    df_output_year = df_output_year.set_index('ccode')
    df_output_year = df_output_year.groupby('ccode').sum()
    
    return df_output_year
create_year_df(1959)   

,res__571.0,res__572.0,res__573.0,res__574.0,res__575.0,res__576.0,res__577.0,res__578.0,res__579.0,res__580.0,...,res__617.0,res__618.0,res__619.0,res__620.0,res__621.0,res__622.0,res__623.0,res__624.0,res__625.0,res__626.0
ccode,,,,,,,,,,,,,,,,,,,,,
2,1,1,0,0,1,1,1,0,0,1,...,0,1,0,0,0,0,1,1,0,0
20,1,1,0,0,1,1,1,0,0,1,...,0,1,0,0,0,0,1,1,0,0
40,1,1,0,0,0,0,0,1,1,0,...,1,1,1,1,1,1,0,0,1,1
41,1,1,0,0,1,1,1,1,0,1,...,1,1,1,1,1,1,0,0,1,1
42,1,1,0,0,1,1,1,0,0,1,...,0,1,0,0,0,0,1,1,0,0
70,1,1,0,0,0,1,1,1,0,0,...,1,1,1,1,1,1,0,0,1,1
90,1,0,0,0,1,1,1,0,0,1,...,0,1,0,0,0,0,0,1,0,0
91,1,1,0,0,1,1,1,0,0,1,...,1,1,0,0,0,0,1,1,0,0
92,1,1,0,0,1,1,1,1,0,1,...,0,1,0,0,0,0,1,1,1,1


In [524]:
frames = {}
groups = [1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
          1975, 1976, 1977, 1978, 1978, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1986, 1988, 1989,
          1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
          2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
for grp in groups: 
    frames[grp] = create_year_df(grp)  

In [525]:
frames.keys()

dict_keys([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [526]:
frames.get(2017)

,res__5546.0,res__5547.0,res__5548.0,res__5549.0,res__5550.0,res__5551.0,res__5552.0,res__5553.0,res__5554.0,res__5555.0,...,res__5630.0,res__5631.0,res__5632.0,res__5633.0,res__5634.0,res__5635.0,res__5636.0,res__5637.0,res__5638.0,res__5639.0
ccode,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
31,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
40,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
41,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,1
42,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
51,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
52,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
53,0,0,0,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,1


In [647]:
def output_all(year):
    hdbscn = hdbscan.HDBSCAN(min_cluster_size = 10,
               gen_min_span_tree = True)
    hdbscn.fit(frames.get(year))
    hdbscn_labels = hdbscn.labels_
    hdbscn_n_clusters_ = len(set(hdbscn_labels)) - (1 if -1 in hdbscn_labels else 0)
    
    df_output_year = create_year_df(year)
    df_output_year['cluster'] = hdbscn.labels_
    df_output_year.reset_index(inplace=True)
    df_output_year = pd.merge(df_output_year, df_name_index, on='ccode')
    
    print('Estimated number of clusters: %d' % hdbscn_n_clusters_)
    # print('Homogeneity: %0.6f' % metrics.homogeneity_score(frames.get(year), hdbscn_labels))
    # print('Completeness: %0.6f' % metrics.completeness_score(y, hdbscn_labels))
    print('Silhouette Coefficient: %0.6f' % metrics.silhouette_score(frames.get(year), hdbscn_labels))
    
    cluster0_list = df_output_year[df_output_year.cluster == 0]['Countryname'].tolist()
    cluster1_list = df_output_year[df_output_year.cluster == 1]['Countryname'].tolist()
    cluster2_list = df_output_year[df_output_year.cluster == 2]['Countryname'].tolist()
    cluster_Outlier_list = df_output_year[df_output_year.cluster == -1]['Countryname'].tolist()
    #df = pd.DataFrame.from_dict(cluster0_list)

    print ("\n\n"," Cluster 0:",cluster0_list,"\n\n","Cluster 1:",cluster1_list,"\n\n","Cluster 2:",cluster2_list,"\n\n","Outliers:",cluster_Outlier_list)
    #return df
    #return df_output_year
    
output_all(1965)  

Estimated number of clusters: 2
Silhouette Coefficient: 0.213032


  Cluster 0: ['Cuba', 'Jamaica', 'Poland', 'Hungary', 'Czechoslovakia', 'Yugoslavia', 'Bulgaria', 'Romania', 'Russian Federation', 'Ukraine', 'Belarus', 'Mali', 'Mauritania', 'Niger', "Côte D'Ivoire", 'Guinea', 'Liberia', 'Sierra Leone', 'Ghana', 'Cameroon', 'Nigeria', 'Congo', 'Uganda', 'Kenya', 'United Republic of Tanzania', 'Rwanda', 'Somalia', 'Ethiopia', 'Zambia', 'Morocco', 'Algeria', 'Libya', 'Sudan', 'Iraq', 'Egypt', 'Syrian Arab Republic', 'Lebanon', 'Jordan', 'Saudi Arabia', 'Yemen Arab Republic', 'Kuwait', 'Afghanistan', 'Mongolia', 'Pakistan', 'Nepal'] 

 Cluster 1: ['United States of America', 'Canada', 'Dominican Republic', 'Mexico', 'Guatemala', 'Honduras', 'El Salvador', 'Costa Rica', 'Panama', 'Colombia', 'Venezuela, Bolivarian Republic of', 'Peru', 'Brazil', 'Bolivia (Plurinational State of)', 'Paraguay', 'Chile', 'Argentina', 'Ireland', 'Netherlands', 'Belgium', 'Luxembourg', 'Spain', 'Austria', 'Ital

In [ ]:
# Create a dummy variable for each yes-vote/resolution combination
# and concatinate with the previous df_1960_dummies dataframe
# Now need to drop the res_0.0 column and group_by.sum() the df
df_1960_dummies = pd.concat([df_1960_features,pd.get_dummies(df_1960_features.v_res, prefix='res_')], axis = 1)

In [ ]:
df_1960_dummies = df_1960_dummies.set_index(['ccode','year',"v_res"])
df_1960_dummies.head()

In [ ]:
#df_t = df_t.drop(['v_res', 'res__0.0', 'vote','member','rcid'], axis=1)
df_1960_dummies = df_1960_dummies.drop( [ 'vote','member','rcid'], axis=1)


In [ ]:
df_1960_dummies.tail()

In [ ]:
df_t = df_1960_dummies.groupby('ccode').sum()
df_t.shape

In [ ]:
features = list(df_t)

In [ ]:
df_t[features].head()

# HDBSCAN

In [ ]:
hdbscn = hdbscan.HDBSCAN(min_cluster_size = 10,
               gen_min_span_tree = True)

In [ ]:
hdbscn.fit(df_t[features])

In [ ]:
hdbscn_labels = hdbscn.labels_

In [ ]:
hdbscn_labels 

In [ ]:
hdbscn_n_clusters_ = len(set(hdbscn_labels)) - (1 if -1 in hdbscn_labels else 0)

In [ ]:
print('Estimated number of clusters: %d' % hdbscn_n_clusters_)
# print('Homogeneity: %0.6f' % metrics.homogeneity_score(y, hdbscn_labels))
# print('Completeness: %0.6f' % metrics.completeness_score(y, hdbscn_labels))
print('Silhouette Coefficient: %0.6f' % metrics.silhouette_score(df_t[features], hdbscn_labels))

In [ ]:
# Need to assign cluster to rows
# Need to add results into visualization
# Need to find a good way to update the year...may have to build out larger data set

In [ ]:
df_t[features].head()

In [ ]:
df_t['cluster'] = hdbscn.labels_

In [ ]:
df_t.head()

In [ ]:
df_votes.head()

In [ ]:
df_t.reset_index(inplace=True)
df_t.head()

In [ ]:
df_votes[['ccode','Countryname']].head()

In [ ]:
df_name_index = pd.DataFrame(df_votes.groupby(['ccode', 'Countryname'])['Countryname'].count())

In [ ]:
df_name_index['count'] = df_name_index.Countryname
df_name_index = df_name_index.drop([ 'Countryname'], axis=1)
df_name_index.head()

In [ ]:
df_name_index.reset_index(inplace=True)
#df_name_index.tail(100)

In [ ]:

df_votes_1960 = pd.merge(df_t, df_name_index, on='ccode')
df_votes_1960.head()

In [ ]:
df_votes.to_csv('df_votes.csv', index=False)

In [ ]:
df_t.to_csv('df_t.csv', index=False)

In [ ]:
#let's see which countres are in the two clusters

In [ ]:
df_votes_1960[df_votes_1960.cluster == 1]['Countryname'] 

In [ ]:
 df_votes_1960.to_csv('df_votes_1960.csv', index=False)

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
df_PCA_1960 = df_votes_1960.drop(['count', 'Countryname'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
#df_votes_1960.set_index('ccode')
df_PCA_1960.head(2)

In [ ]:
X = df_PCA_1960

pca = PCA()
pca = pca.fit(X)
X

In [ ]:
Z = pca.transform(X)
features_pca = ['PC'+str(i+1) for i in range(pca.n_components_)]
Z = pd.DataFrame(Z, columns=features_pca)
Z.head()

In [ ]:
print(pca.explained_variance_)

In [ ]:
abs((pca.components_))[0]

In [ ]:

print(abs((pca.components_))[0], '\n')
#print('-------------------------------------\n')



In [ ]:
X.reset_index(inplace=True)
X.head()

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
#pd.concat([X,Z])
X = X.join(Z, lsuffix='_caller', rsuffix='_other')
X

In [ ]:
df_PCA_1960 = pd.merge(X, df_PCA_1960, on='ccode')

In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['res__627.0', 'res__628.0', 'res__629.0', 'res__630.0', 'res__631.0', 
                            'res__632.0', 'res__633.0', 'res__634.0', 'res__635.0', 'res__636.0', 
                            'res__637.0', 'res__638.0', 'res__639.0', 'res__640.0', 'res__641.0', 
                            'res__642.0', 'res__643.0', 'res__644.0', 'res__645.0', 'res__646.0', 
                            'res__647.0', 'res__648.0', 'res__649.0', 'res__650.0', 'res__651.0', 
                            'res__652.0', 'res__653.0', 'res__654.0', 'res__655.0', 'res__656.0', 
                            'res__657.0', 'res__658.0', 'res__659.0', 'res__660.0', 'res__661.0', 
                            'res__662.0', 'res__663.0', 'res__664.0', 'res__665.0', 'res__666.0', 
                            'res__667.0', 'res__668.0', 'res__669.0', 'res__670.0', 'res__671.0', 
                            'res__672.0', 'res__673.0', 'res__674.0', 'res__675.0', 'res__676.0', 
                            'res__677.0', 'res__678.0', 'res__9040.0', 'res__9041.0'], axis=1)


In [ ]:
df_PCA_1960 = df_PCA_1960.drop(['index_x','PC3', 'PC4', 'PC5',
       'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14',
       'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23',
       'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32',
       'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41',
       'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50',
       'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56', 'index_y', 'cluster_y'], axis=1)

In [ ]:
df_PCA_1960.head(2)

In [ ]:
fig = plt.figure(figsize=(9,7))

ax = fig.gca()
ax.scatter(Z.iloc[:,0], Z.iloc[:,1],
                   c=df_PCA_1960.cluster_x)

ax.set_xlabel('principal component 1', fontsize=16)
ax.set_ylabel('principal component 2', fontsize=16)


for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
ax.set_title('PC1 vs PC2\n', fontsize=20)

plt.show()

In [ ]:
df_PCA_1960.head(2)

# NLP

In [ ]:

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.patches as mpatches
import matplotlib


In [656]:
df_descriptions = pd.read_csv('~/documents/UN-Capstone/Dscription.csv')

In [659]:
df_descriptions.head(0)

,Unnamed: 0,session,rcid,abstain,yes,no,importantvote,date,unres,amend,para,short,descr,me,nu,di,hr,co,ec,year


In [660]:
texts = []

In [662]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- one small tweak

In [663]:
for i in range(1000):
    texts.append(text_process(df_descriptions.loc[i, 'descr']))

NameError: name 'RegexpTokenizer' is not defined

In [ ]:
df_UNAid = pd.read_csv('~/documents/UN-Capstone/net_aid.csv')

In [ ]:
df_UNAid

In [ ]:
#     fmt = '%-8s%-40s%s'

#     print(fmt % ('', 'Cluster 0', 'Cluster 1'))
#     for i, (name, grade) in enumerate(zip(cluster0_list,cluster1_list)):
#         print(fmt % (i, name, grade))